In [1]:
import torch
import numpy as np

c:\Users\hiren\Anaconda3\envs\yolo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Implement Alexnet model
class Alexnet(torch.nn.Module):
    def __init__(self,in_channels = 3,num_classes = 1000):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels = in_channels,out_channels = 96,kernel_size = 11,stride = 4,padding = 0)
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size = 3,stride = 2,padding = 0)
        self.normlocal = torch.nn.LocalResponseNorm(size = 5,alpha = 0.0001,beta = 0.75,k = 2)
        self.conv2 = torch.nn.Conv2d(in_channels = 96,out_channels = 256,kernel_size = 5,stride = 1,padding = 2)
        self.conv3 = torch.nn.Conv2d(in_channels = 256,out_channels = 384,kernel_size = 3,stride = 1,padding = 1)
        self.conv4 = torch.nn.Conv2d(in_channels = 384,out_channels = 384,kernel_size = 3,stride = 1,padding = 1)
        self.conv5 = torch.nn.Conv2d(in_channels = 384,out_channels = 256,kernel_size = 3,stride = 1,padding = 1)
        self.dense1 = torch.nn.Linear(in_features = 9216,out_features = 4096)
        self.dense2 = torch.nn.Linear(in_features = 4096,out_features = 4096)
        self.dense3 = torch.nn.Linear(in_features = 4096,out_features = num_classes)
        self.dropout = torch.nn.Dropout(p = 0.5)
        self.flatten = torch.nn.Flatten()
        self.relu = torch.nn.ReLU()
        # 
    # def OverlappingMaxPool2d(kernel_size,stride,padding):
        
        
    
    # def forward(self,x):
    #     result = self.conv1(x)
    #     result = self.relu(result)
    #     result = self.normlocal(result)
    #     result = self.maxpool1(result)
    #     result = self.conv2(result)
    #     result = self.relu(result)
    #     result = self.normlocal(result)
    #     result = self.maxpool1(result)
    #     result = self.conv3(result)
    #     result = self.relu(result)
    #     result = self.conv4(result)
    #     result = self.relu(result)
    #     result = self.conv5(result)
    #     result = self.relu(result)
    #     result = self.maxpool1(result)
    #     # result = self.flatten(result)
    #     # result = result.view(result.size(0),-1)
    #     result = self.flatten(result)
    #     result = self.dense1(result)
    #     result = self.relu(result)
    #     result = self.dropout(result)
    #     result = self.dense2(result)
    #     result = self.relu(result)
    #     result = self.dropout(result)
    #     result = self.dense3(result)
    #     result = torch.nn.functional.softmax(result,dim = 1)
    #     return result
    
    def forward(self,x):
        return torch.nn.functional.softmax(self.dense3(self.dropout(self.relu(self.dense2(self.dropout(
            self.relu(self.dense1(self.flatten(self.maxpool1(self.relu(self.conv5(self.relu(
                self.conv4(self.relu(self.conv3(self.maxpool1(self.normlocal(self.relu(self.conv2(
                    self.maxpool1(self.normlocal(self.relu(self.conv1(x))))))))))))))))))))))),dim = 1)


In [3]:
import torchvision
from torchvision import transforms

transform_data = torchvision.transforms.Compose([
    # transforms.RandomResizedCrop(227, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    # transforms.RandomHorizontalFlip(),
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406],std = [0.229,0.224,0.225])
])

train_dataset = torchvision.datasets.CIFAR10(root = 'data',train = True,transform = transform_data,download = True)
test_dataset = torchvision.datasets.CIFAR10(root = 'data',train = False,transform = transform_data,download = True)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
from tqdm import tqdm
model = Alexnet(in_channels = 3,num_classes = 10).cuda()
train_data_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 156,shuffle = True)
test_data_loader = torch.utils.data.DataLoader(test_dataset,batch_size = 156,shuffle = True)
train_data_loader = test_data_loader
epochs = 100
learning_rate = 0.00001
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size = 10,gamma = 0.1)
# total_train_batches = 

# def f5_accuracy 

for epoch in tqdm(range(epochs)):
    train_loss = 0
    test_loss = 0
    train_acc = 0
    test_acc = 0
    f1 = 0
    f5 = 0
    
    for batch, (x,y) in enumerate(tqdm(train_data_loader)):
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (y_pred.argmax(1) == y).type(torch.float).mean().item()
        
    with torch.no_grad():
        for batch, (x,y) in enumerate(tqdm(test_data_loader)):
            x = x.cuda()
            y = y.cuda()
            y_pred = model(x)
            loss = loss_fn(y_pred,y)
            test_loss += loss.item()
            test_acc += (y_pred.argmax(1) == y).type(torch.float).mean().item()
            f1 += (y_pred.argmax(1) == y).type(torch.float).mean().item()
            
    
    print('Epoch: {}, Train Loss: {}, Train Acc: {}, Test Loss: {}, Test Acc: {}'.format(epoch,train_loss/len(train_data_loader),train_acc/len(train_data_loader),test_loss/len(test_data_loader),test_acc/len(test_data_loader)))

  1%|          | 1/100 [01:58<3:16:02, 118.81s/it]

Epoch: 0, Train Loss: 2.2981041321387656, Train Acc: 0.12946252822875975, Test Loss: 2.2730896693009597, Test Acc: 0.15894970848010137


  2%|▏         | 2/100 [03:53<3:09:37, 116.09s/it]

Epoch: 1, Train Loss: 2.2273879784804125, Train Acc: 0.22411242998563327, Test Loss: 2.1913400870103104, Test Acc: 0.2539447763791451


  3%|▎         | 3/100 [05:46<3:06:02, 115.07s/it]

...

 38%|███▊      | 38/100 [1:16:17<2:07:05, 123.00s/it]

Epoch: 37, Train Loss: 2.0023802243746243, Train Acc: 0.4550049405831557, Test Loss: 2.011353892546434, Test Acc: 0.4450936977679913


 39%|███▉      | 39/100 [1:18:31<2:08:23, 126.29s/it]

Epoch: 38, Train Loss: 1.9962114370786226, Train Acc: 0.4667406375591572, Test Loss: 1.9900926773364727, Test Acc: 0.47243590354919435


100%|██████████| 65/65 [01:20<00:00,  1.23s/it]
